___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Combining DataFrames

## Full Official Guide (Lots of examples!)

### https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

-------
-------

In [1]:
import numpy as np
import pandas as pd

## Concatenation

Directly  "glue" together dataframes.

In [2]:
data_one = {'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']}

In [3]:
data_two = {'C': ['C0', 'C1', 'C2', 'C3'], 'D': ['D0', 'D1', 'D2', 'D3']}

In [4]:
one = pd.DataFrame(data_one)

In [5]:
two = pd.DataFrame(data_two)

In [6]:
one

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [7]:
two

,C,D
0,C0,D0
1,C1,D1
2,C2,D2
3,C3,D3


## Axis = 0 

### Concatenate along rows

In [8]:
axis0 = pd.concat([one,two],axis=0)

In [9]:
axis0

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,A3,B3,NaN,NaN
0,NaN,NaN,C0,D0
1,NaN,NaN,C1,D1
2,NaN,NaN,C2,D2
3,NaN,NaN,C3,D3


## Axis = 1

### Concatenate along columns

In [10]:
axis1 = pd.concat([one,two],axis=1)

In [11]:
axis1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


### Axis 0 , but columns match up
**In case you wanted this:**

In [12]:
two.columns = one.columns


# Merge

## Data Tables

In [13]:
registrations = pd.DataFrame({'reg_id':[1,2,3,4],'name':['Andrew','Bobo','Claire','David']})
logins = pd.DataFrame({'log_id':[1,2,3,4],'name':['Xavier','Andrew','Yolanda','Bobo']})

In [14]:
pd.merge(left=registrations,right=logins,on="name",how="inner")

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [15]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


# pd.merge()

Merge pandas DataFrames based on key columns, similar to a SQL join. Results based on the **how** parameter.

In [16]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | AnyArrayLike | None' = None, left_on: 'IndexLabel | AnyArrayLike | None' = None, right_on: 'IndexLabel | AnyArrayLike | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.

    A named Series object is treated as a DataFrame with a single named column.

    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.

  

-----

# Inner,Left, Right, and Outer Joins

## Inner Join

**Match up where the key is present in BOTH tables. There should be no NaNs due to the join, since by definition to be part of the Inner Join they need info in both tables.**
**Only Andrew and Bobo both registered and logged in.**

In [17]:
# Notice pd.merge doesn't take in a list like concat
pd.merge(registrations,logins,how='inner',on='name')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [18]:
# Pandas smart enough to figure out key column (on parameter) if only one column name matches up
pd.merge(registrations,logins,how='inner')

,reg_id,name,log_id
0,1,Andrew,2
1,2,Bobo,4


In [19]:
# Pandas reports an error if "on" key column isn't in both dataframes
# pd.merge(registrations,logins,how='inner',on='reg_id')

---

## Left Join

**Match up AND include all rows from Left Table.**
**Show everyone who registered on Left Table, if they don't have login info, then fill with NaN.**

In [20]:
pd.merge(registrations,logins,how='left')

,reg_id,name,log_id
0,1,Andrew,2.0
1,2,Bobo,4.0
2,3,Claire,NaN
3,4,David,NaN


## Right Join
**Match up AND include all rows from Right Table.**
**Show everyone who logged in on the Right Table, if they don't have registration info, then fill with NaN.**

In [44]:
pd.merge(left = logins,right =registrations,how='right')

,id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


## Outer Join

**Match up on all info found in either Left or Right Table.**
**Show everyone that's in the Log in table and the registrations table. Fill any missing info with NaN**

In [22]:
pd.merge(registrations,logins,how='outer')

,reg_id,name,log_id
0,1.0,Andrew,2.0
1,2.0,Bobo,4.0
2,3.0,Claire,NaN
3,4.0,David,NaN
4,NaN,Xavier,1.0
5,NaN,Yolanda,3.0


## Join on Index or Column

**Use combinations of left_on,right_on,left_index,right_index to merge a column or index on each other**

In [23]:
registrations

,reg_id,name
0,1,Andrew
1,2,Bobo
2,3,Claire
3,4,David


In [24]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [25]:
registrations = registrations.set_index("name")

In [26]:
registrations

,reg_id
name,
Andrew,1
Bobo,2
Claire,3
David,4


In [27]:
pd.merge(registrations,logins,left_index=True,right_on='name')

,reg_id,log_id,name
1,1,2,Andrew
3,2,4,Bobo


In [28]:
pd.merge(logins,registrations,right_index=True,left_on='name')

,log_id,name,reg_id
1,2,Andrew,1
3,4,Bobo,2


### Dealing with differing key column names in joined tables

In [29]:
registrations = registrations.reset_index()

In [30]:
registrations

,name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [31]:
logins

,log_id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [32]:
registrations.columns = ['reg_name','reg_id']

In [33]:
registrations

,reg_name,reg_id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [34]:
# ERROR
# pd.merge(registrations,logins)

In [35]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name')

,reg_name,reg_id,log_id,name
0,Andrew,1,2,Andrew
1,Bobo,2,4,Bobo


In [36]:
pd.merge(registrations,logins,left_on='reg_name',right_on='name').drop('reg_name',axis=1)

,reg_id,log_id,name
0,1,2,Andrew
1,2,4,Bobo


### Pandas automatically tags duplicate columns

In [37]:
registrations.columns = ['name','id']

In [38]:
logins.columns = ['id','name']

In [39]:
registrations

,name,id
0,Andrew,1
1,Bobo,2
2,Claire,3
3,David,4


In [40]:
logins

,id,name
0,1,Xavier
1,2,Andrew
2,3,Yolanda
3,4,Bobo


In [41]:
# _x is for left
# _y is for right
pd.merge(registrations,logins,on='name')

,name,id_x,id_y
0,Andrew,1,2
1,Bobo,2,4


In [42]:
pd.merge(registrations,logins,on='name',suffixes=('_reg','_log'))

,name,id_reg,id_log
0,Andrew,1,2
1,Bobo,2,4


-----------
----------